In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/prodata2/xxx.csv', usecols=['type', 'text'])
# name: full viet nam co dau
# address: mix vietnam - usa 
# web: random website
# can them dataset nhieu hon

**Created by Peter Nagy | 2018 May**<br/>
 [Github](https://github.com/nagypeterjob)  <br/>
 [Linkedin](https://www.linkedin.com/in/peternagyjob/)<br/>
In this kernel I do perform a multi-class classification with LSTM (Keras).

In [ ]:
#M class has way less data than the orthers, thus the classes are unbalanced.
data.type.value_counts()

In [ ]:
#I do aspire here to have balanced classes
num_of_categories = 45000
shuffled = data.reindex(np.random.permutation(data.index))
name = shuffled[shuffled['type'] == 'name'][:num_of_categories]
address = shuffled[shuffled['type'] == 'address'][:num_of_categories]
website = shuffled[shuffled['type'] == 'website'][:num_of_categories]
company = shuffled[shuffled['type'] == 'company'][:num_of_categories]
engname = shuffled[shuffled['type'] == 'engname'][:num_of_categories]
concated = pd.concat([name,address,website,company, engname], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

In [ ]:
#One-hot encode the lab
concated.loc[concated['type'] == 'name', 'LABEL'] = 0
concated.loc[concated['type'] == 'address', 'LABEL'] = 1
concated.loc[concated['type'] == 'website', 'LABEL'] = 2
concated.loc[concated['type'] == 'company', 'LABEL'] = 3
concated.loc[concated['type'] == 'engname', 'LABEL'] = 4
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=5)
print(labels[:10], "...")
if 'type' in concated.keys():
    concated.drop(['type'], axis=1)
'''
 [1. 0. 0. 0.] e
 [0. 1. 0. 0.] b
 [0. 0. 1. 0.] t

'''

In [ ]:
n_most_common_words = 20000
max_len = 130

tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-;<=>?@[\]^_`{|}~', lower=True) # ./-:
#print(concated['text'].values
texts =concated['text'].values.astype("str")
for i in range (0,len(texts)):
    texts[i]=texts[i].astype("str").replace("."," ").replace("/"," ").replace("-"," ").replace(":"," ")

print(texts[:10])
tokenizer.fit_on_texts(texts)
#tokenizer.fit_on_texts(concated['text'].values)
sequences = tokenizer.texts_to_sequences(concated['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=42)

In [ ]:
epochs = 30
emb_dim = 128
batch_size = 1024
print(labels[24])
print(X[24])
#print(tokenizer.word_index)


#name,address,website,company, engname

In [ ]:
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())
#history = model.fit(X_train, y_train, epochs=20, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#model.save("yy.h5")
import keras
model = keras.models.load_model("../input/ewwwww/eww.h5")
txt = ["john mcdonnel"]
print(txt)
seq = tokenizer.texts_to_sequences(txt)


print(seq)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = ['name', 'address', 'web', 'company', 'eng-name']

np_array = np.array( pred)
np_round_to_tenths = np.around(np_array, 2)
round_to_tenths = list(np_round_to_tenths)

print(np.argsort(np.max(np_array, axis=0))[-2])
print(np.argmax(pred))

print(round_to_tenths, labels[np.argmax(pred)])










